# Рекомендательные системы

КАК ПОЛУЧИТЬ ЗАЧЁТ
1. установить mrec
2. получить для MovieLens 100K рекомендации:
    - по популярности 
    - с помощью kNN
3. поделиться результатами с преподавателем

КАКИЕ ИМЕННО РЕЗУЛЬТАТЫ НУЖНЫ
1. файлы, сгенерированные скриптами:
    - mrec_prepare (train/test)
    - mrec_train (model/similarity)
    - mrec_predict (собственно рекомендации)
2. отчёт об исследовании
    - какие скрипты вызывали, в каком порядке 
    - какие модели пробовали
    - какие параметры использовали    

КАК ПОЛУЧИТЬ УДОВОЛЬСТВИЕ
1. используйте продвинутые модели 
    - SLIM
    - WRMF (подробнее в лекции 4.3) 
    - WARP (подробнее в лекции 4.6)
2. подберите параметры моделей
    - k и меру схожести в kNN
    - параметры регуляризации в SLIM 
    - alpha и lambda в WRMF    

КАК СТАТЬ DATA SCIENTIST’ОМ
- сделать fork репы и добавить код,
    - считающий Recall
    - или считающий MAP (mean average precision) 
    - метрику на свой выбор
- запустить обучение в облаке в 20 потоков

### Погнали, как получить зачет

Установим библиотеку

```git clone https://github.com/inpefess/mrec
cd mrec
sudo -H python3 setup.py install --install-scripts sbin```

Положим папочку с датасетом рядом и импортируем его 

In [141]:
import sys
import pandas as pd
import scipy
import mrec

In [142]:
from os import path

data_dir = "ml-latest-small"

data = pd.read_csv("ml-100k/u.data", sep="\t", header=None, index_col=0)
data.columns = ["item id" , "rating" , "timestamp"]

Посмотрим оценки

In [143]:
data.head()

,item id,rating,timestamp
0,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


про фильмы знаем названия и жанры

In [144]:
item = pd.read_csv("ml-100k/u.item", sep="|", header=None, index_col=None,  encoding = "ISO-8859-1")
item.columns = ["movie_id", "movie title" , "release date" , "video release date" ,
              "IMDb URL" , "unknown" , "Action" , "Adventure" , "Animation" ,
              "Children's" , "Comedy" ,"Crime" ,"Documentary" , "Drama" , "Fantasy" ,
              "Film-Noir" , "Horror" , "Musical" , "Mystery" , "Romance" , "Sci-Fi" ,
              "Thriller" , "War" , "Western"]
item.head()

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


#### Подготовим данные для тренировки

In [161]:
!sbin/mrec_prepare --dataset ml-100k/u.data --outdir splits --rating_thresh 4 --test_size 0.5 --binarize

[2017-11-22 17:42:35,164] INFO: sorting input data...
[2017-11-22 17:42:36,345] INFO: creating split 0: /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.train.0 /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.test.0
[2017-11-22 17:42:36,936] INFO: creating split 1: /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.train.1 /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.test.1
[2017-11-22 17:42:37,721] INFO: creating split 2: /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.train.2 /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.test.2
[2017-11-22 17:42:38,455] INFO: creating split 3: /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.train.3 /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.test.3
[2017-11-22 17:42:39,298] INFO: creating split 4: /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.train.4 /Users/vladimir/temp/AnacondaProje

попробуем выполнить все в несколько потоков

In [162]:
!sbin/ipcluster start -n4 --daemonize

## SLIM

### Начнем обучаться

In [163]:
!sbin/mrec_train -n4 --input_format tsv --train "splits/u.data.train.*" --outdir models

/Users/vladimir/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
[2017-11-22 17:42:49,369] INFO: processing /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.train.0...
[2017-11-22 17:42:49,369] INFO: finding number of items...
[2017-11-22 17:42:50,018] INFO: 943 users and 1678 items
[2017-11-22 17:42:50,018] INFO: creating sims directory /Users/vladimir/temp/AnacondaProjects/homework/mrec/models/u.data.train.0-sims...
[2017-11-22 17:42:50,131] INFO: checking for existing output sims...
[2017-11-22 17:42:50,133] INFO: creating tasks...
[2017-11-22 17

### Предсказание

In [164]:
!sbin/mrec_predict --input_format tsv --test_input_format tsv --train "splits/u.data.train.*" --modeldir models --outdir recs

[2017-11-22 17:43:44,276] INFO: processing /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.train.0...
[2017-11-22 17:43:44,276] INFO: creating recs directory /Users/vladimir/temp/AnacondaProjects/homework/mrec/recs/u.data.train.0-recs...
[2017-11-22 17:43:44,302] INFO: checking for existing output recs...
[2017-11-22 17:43:44,303] INFO: creating tasks...
[2017-11-22 17:43:44,303] INFO: loading dataset to get size...
[2017-11-22 17:43:44,790] INFO: loading model to get size...
[2017-11-22 17:43:45,276] INFO: created 1 tasks, 21272 users per task
[2017-11-22 17:43:45,276] INFO: running in parallel across ipython engines...
[2017-11-22 17:43:48,414] INFO: checking output files...
[2017-11-22 17:43:48,414] INFO: SUCCESS: all tasks completed
[2017-11-22 17:43:48,414] INFO: concatenating 1 partial output files...
[2017-11-22 17:43:48,435] INFO: removing partial output files...
[2017-11-22 17:43:48,436] INFO: done
[2017-11-22 17:43:48,438] INFO: processing /Users/vladimir/te

## Метод ближайших соседей (KNN)

### Начнем обучаться

In [149]:
!sbin/mrec_train --model=knn -n4 --input_format tsv --train "splits/u.data.train.*" --outdir models_knn

[2017-11-22 17:38:16,062] INFO: processing /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.train.0...
[2017-11-22 17:38:16,062] INFO: finding number of items...
[2017-11-22 17:38:16,591] INFO: 943 users and 1682 items
[2017-11-22 17:38:16,591] INFO: creating sims directory /Users/vladimir/temp/AnacondaProjects/homework/mrec/models_knn/u.data.train.0-sims...
[2017-11-22 17:38:16,601] INFO: checking for existing output sims...
[2017-11-22 17:38:16,603] INFO: creating tasks...
[2017-11-22 17:38:16,603] INFO: running 4 tasks in parallel across ipython engines...
[2017-11-22 17:38:19,577] INFO: checking output files...
[2017-11-22 17:38:19,577] INFO: SUCCESS: all tasks completed
[2017-11-22 17:38:19,577] INFO: concatenating 4 partial output files...
[2017-11-22 17:38:19,612] INFO: removing partial output files...
[2017-11-22 17:38:19,614] INFO: loading 1682 items in CosineKNNRecommender model from /Users/vladimir/temp/AnacondaProjects/homework/mrec/models_knn/u.data.train.

### Предсказание

In [150]:
!sbin/mrec_predict --input_format tsv --test_input_format tsv --train "splits/u.data.train.*" --modeldir models_knn --outdir recs_knn

[2017-11-22 17:38:46,173] INFO: processing /Users/vladimir/temp/AnacondaProjects/homework/mrec/splits/u.data.train.0...
[2017-11-22 17:38:46,173] INFO: creating recs directory /Users/vladimir/temp/AnacondaProjects/homework/mrec/recs_knn/u.data.train.0-recs...
[2017-11-22 17:38:46,197] INFO: checking for existing output recs...
[2017-11-22 17:38:46,198] INFO: creating tasks...
[2017-11-22 17:38:46,199] INFO: loading dataset to get size...
[2017-11-22 17:38:46,713] INFO: loading model to get size...
[2017-11-22 17:38:46,988] INFO: created 1 tasks, 10823 users per task
[2017-11-22 17:38:46,988] INFO: running in parallel across ipython engines...
[2017-11-22 17:38:50,334] INFO: checking output files...
[2017-11-22 17:38:50,334] INFO: SUCCESS: all tasks completed
[2017-11-22 17:38:50,334] INFO: concatenating 1 partial output files...
[2017-11-22 17:38:50,377] INFO: removing partial output files...
[2017-11-22 17:38:50,377] INFO: done
[2017-11-22 17:38:50,379] INFO: processing /Users/vladimi

Посмотрим на то что вышло

In [165]:
data_pred = pd.read_csv("recs/u.data.train.0.recs.tsv", sep="\t", header=None, index_col=None)
data_pred.columns = ["user_id", "movie_id", "pred"]

In [166]:
data_pred.head()

,user_id,movie_id,pred
0,1,50,0.570405
1,1,183,0.528868
2,1,181,0.520021
3,1,168,0.499544
4,1,82,0.479166


In [167]:
mo = data_pred.head()["movie_id"]
mo

0     50
1    183
2    181
3    168
4     82
Name: movie_id, dtype: int64

In [168]:
ind = item[item["movie_id"].isin(mo.values)]["movie title"]
ind

49                           Star Wars (1977)
81                       Jurassic Park (1993)
167    Monty Python and the Holy Grail (1974)
180                 Return of the Jedi (1983)
182                              Alien (1979)
Name: movie title, dtype: object